In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10, FashionMNIST
from torch.utils.data import DataLoader

/home/ubuntu-user/anaconda3/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [2]:
# 定义深度卷积神经网络模型
class Model(nn.Module):
    def __init__(self, num_classes):
        super(Model, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.classifier = nn.Sequential(
            #nn.Linear(64 * 16 * 16, 128),
            nn.Linear(64 * 16 * 16, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )
        self.classifier1 = nn.Sequential(
            #nn.Linear(64 * 16 * 16, 128),
            nn.Linear(64 * 14 * 14, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )
        self.unknown_class = num_classes

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

    def open_set_forward(self, x, known_classes):
        x = self.features(x)
        #print(x.shape) 
        x = x.view(x.size(0), -1)
        #print(x.shape) 
        logits = self.classifier1(x)

        # 计算每个样本的开放集得分
        open_scores = torch.softmax(logits[:, known_classes], dim=1)

        # 添加未知类别得分
        unknown_scores = torch.zeros(x.size(0), 1).to(x.device)
        open_scores = torch.cat((open_scores, unknown_scores), dim=1)

        return open_scores

In [3]:
epoch_number=[]
loss_record=[]

In [4]:
class_number=[]
class_acc=[]

In [5]:
# 训练闭集模型
def train(model, train_loader, criterion, optimizer, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        epoch_number.append(epoch)
        running_loss = 0.0
        for i, (images, labels) in enumerate(train_loader):
            images = images.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        loss_record.append(running_loss / len(train_loader))
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(train_loader)}")

In [6]:
# 测试开放集识别
def test(model, test_loader, known_classes):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)

            open_scores = model.open_set_forward(images, known_classes)
            _, predicted = torch.max(open_scores.data, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100.0 * correct / total
    print(f"Test Accuracy: {accuracy}%")

In [7]:
# 设置随机种子和设备
torch.manual_seed(2024)
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")


In [8]:
# 数据预处理和加载器
'''
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((32, 32)),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
'''

'\ntransform = transforms.Compose([\n    transforms.ToTensor(),\n    transforms.Resize((32, 32)),\n    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))\n])\n'

In [9]:

# Define the transformations for CIFAR10
transform_cifar = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((32, 32)),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


In [10]:
# Define the transformations for MNIST
transform_mnist = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Resize((32, 32)),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [11]:
train_dataset = torchvision.datasets.CIFAR10(root='data', train=True, download=True, transform=transform_cifar)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)

Files already downloaded and verified


In [12]:
# 加载CIFAR10测试集
testset = torchvision.datasets.CIFAR10(root='data', train=False,
                                       download=True, transform=transform_cifar)
testloader = DataLoader(testset, batch_size=64,
                                         shuffle=False, num_workers=2)


Files already downloaded and verified


In [13]:
test_dataset = torchvision.datasets.FashionMNIST(root='data', train=False, download=True, transform=transform_mnist)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)


In [14]:
# 初始化模型、损失函数和优化器
num_classes = 10  # 已知类别数量
'''
model = Model(num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())
'''


'\nmodel = Model(num_classes).to(device)\ncriterion = nn.CrossEntropyLoss()\noptimizer = optim.Adam(model.parameters())\n'

In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models

# 加载预训练的ResNet18模型
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)


/home/ubuntu-user/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/ubuntu-user/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
# 训练模型
train(model, train_loader, criterion, optimizer, num_epochs=10)

Epoch [1/10], Loss: 1.0360290232064473
Epoch [2/10], Loss: 0.6318752801479282
Epoch [3/10], Loss: 0.4783939341526202
Epoch [4/10], Loss: 0.3774367943215553
Epoch [5/10], Loss: 0.28748140018195145
Epoch [6/10], Loss: 0.22495719534166328
Epoch [7/10], Loss: 0.17330804607495093
Epoch [8/10], Loss: 0.1474898133612693
Epoch [9/10], Loss: 0.11830203011250862


In [ ]:

class_correct = [0] * 10
class_total = [0] * 10

with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(len(labels)):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

for i in range(10):
    class_number.append(i)
    accuracy = 100 * class_correct[i] / class_total[i]
    class_acc.append(accuracy)
    print(f'Accuracy of class {i}: {accuracy:.2f}%')

In [ ]:
import numpy as np
from sklearn.metrics import roc_curve, auc

# 计算开放集似然性
def calculate_open_set_likelihood(testloader, open_set_loader):
    model.eval()
    softmax = nn.Softmax(dim=1)

    scores = []
    labels = []

    with torch.no_grad():
        for data in testloader:
            images, true_labels = data
            outputs = model(images)
            probabilities = softmax(outputs)
            max_probabilities, _ = torch.max(probabilities, 1)
            scores.extend(max_probabilities.cpu().numpy())
            labels.extend(true_labels.cpu().numpy())

    open_set_scores = []

    with torch.no_grad():
        for data in open_set_loader:
            images, _ = data
            outputs = model(images)
            probabilities = softmax(outputs)
            max_probabilities, _ = torch.max(probabilities, 1)
            open_set_scores.extend(max_probabilities.cpu().numpy())

    return np.array(scores), np.array(open_set_scores), np.array(labels)

# 计算似然性并绘制ROC曲线
scores, open_set_scores, labels = calculate_open_set_likelihood(testloader, test_loader)

fpr, tpr, thresholds = roc_curve(labels, scores, pos_label=1)
roc_auc = auc(fpr, tpr)

open_set_fpr, open_set_tpr, _ = roc_curve(np.concatenate((np.ones_like(labels), np.zeros_like(open_set_scores))), np.concatenate((scores, open_set_scores)))
open_set_roc_auc = auc(open_set_fpr, open_set_tpr)

# 绘制ROC曲线
import matplotlib.pyplot as plt
plt.figure()
plt.plot(fpr, tpr, color='darkorange', label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot(open_set_fpr, open_set_tpr, color='blue', linestyle='--', label='Open-set ROC curve (area = %0.2f)' % open_set_roc_auc)
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

In [ ]:
plt.xlabel('epoch')
plt.ylabel('loss')
plt.title("The change of loss under different epoches")
plt.plot(epoch_number, loss_record, linewidth=2,  marker='.')
#plt.legend()
plt.show()

In [ ]:
plt.xlabel('class')
plt.ylabel('Accuracy')
plt.title("The change of accuracy under different classess")
plt.plot(class_number, class_acc, linewidth=2,  marker='.')
#plt.legend()
plt.show()